# Assignment 4 – Privacy-Preserving Federated Learning Simulation (Breast Cancer Dataset)

### Objective

Simulate federated learning across two “hospitals” using the Breast Cancer Wisconsin (Diagnostic) dataset without sharing raw data. Add a differential-privacy layer, compare against centralized training, and document privacy/utility trade-offs.

## 0. Imports & Setup

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import flwr as fl       #type: ignore

import torch, opacus    # type: ignore